### Import modules and constants

In [7]:
import json
import os
import argparse
from collections import Counter
import csv

baseline_packets_redirected = {"registered": {"Monday": 11709971, "Tuesday": 11551954, "Wednesday": 13788878, "Thursday": 9322025, "Friday": 9997874}}

n_redirected_packets=200
time_threshold=10
count_min_size=16384

### Define the functions that can read the alerts and number of packets redirected from the results

In [13]:
def read_snort_alerts(alert_filepath):
	no_duplicates_data = {}
	data = []
	counter = Counter()
	file = open(alert_filepath)
	if os.path.getsize(alert_filepath) > 0:
		try:
			for line in file.readlines():
				parsed_line = json.loads(line)
				data.append(parsed_line)

				entry_key = str(parsed_line["pkt_num"]) + parsed_line["rule"] + parsed_line["timestamp"]
				if entry_key not in no_duplicates_data:
					no_duplicates_data[entry_key] = line
				
				counter[parsed_line["rule"]]+=1
		except Exception as e:
			print("JSON error: ", e)
	file.close()
	return data, counter, no_duplicates_data

def read_amt_of_redirected_pkts(p4_output_filepath):
	packets_redirected = 0
	try:
		with open(p4_output_filepath) as file:
			packets_redirected = 0
			get_data = False
			previous_line = ""
			for line in file:
				if "Counter name:  MyEgress.cloned_to_ids" in previous_line and "Index (port):  index: 2" in line:
					get_data = True
				
				if "packet_count" in line and get_data==1:
					packets_redirected += int(line.split()[1])
					get_data = False

				previous_line = line
	except:
		print("No output file in this folder")

	return packets_redirected

### Read baseline alerts

In [3]:
baseline_alerts_folder="../../baseline_alerts/CICIDS2017/alerts_snort3-registered"

def read_baseline(baseline_folder):
	baseline_data = {}
	for alert_file in os.listdir(baseline_folder):
		item_fullpath = os.path.join(baseline_folder, alert_file)

		raw_data, rules_counter, no_duplicates_data =  read_snort_alerts(item_fullpath)
		baseline_data[alert_file.split(".")[0]]={"alerts": no_duplicates_data, "counter": rules_counter}
	
	return baseline_data


baseline_data = read_baseline(baseline_alerts_folder)

### Read the results of one complete experiment: Monday, Tuesday, etc.

In [4]:
def read_experiments_data(experiments_data_folder):
	folder_data = {}
	for item in os.listdir(experiments_data_folder):
		item_fullpath = os.path.join(experiments_data_folder, item)
		if os.path.isfile(item_fullpath):
			continue

		no_duplicates_data = {}
		raw_data = []
		rules_counter = Counter()
		for subdir in os.listdir(item_fullpath):
			if os.path.isfile(os.path.join(item_fullpath, subdir)):
				continue

			alert_file = os.path.join(item_fullpath, subdir) + "/alert_json.txt"
			data, counter, no_duplicates =  read_snort_alerts(alert_file)
			raw_data.extend(data)
			rules_counter.update(counter)
			for key, value in no_duplicates.items():
				if key not in no_duplicates_data:
					no_duplicates_data[key] = value

		packets_redirected = read_amt_of_redirected_pkts(os.path.join(item_fullpath, "output.txt"))
		folder_data[item]={"alerts": no_duplicates_data, "counter": rules_counter, "packets_redirected": packets_redirected}
	return folder_data

### Parse the parameters evaluation experiments

In [22]:
input_folder = "../../../experiments/final_evaluation_results"
output_folder = "./"
csv_data = []
for item in os.listdir(input_folder):
    item_fullpath = os.path.join(input_folder, item)
    if os.path.isfile(item_fullpath):
        continue

    folder_name_elements = item.split("_") # linear_simple_100_registered
    experiments_data = read_experiments_data(item_fullpath)

    for key, data in experiments_data.items():
        csv_line = {}

        
        csv_line["PCAP"] = key
        csv_line["Topology"] = folder_name_elements[0]
        csv_line["Offloading algorithm"] = folder_name_elements[1]
        csv_line["Available space (%)"] = folder_name_elements[2]
        csv_line["Alerts"] = len(data["alerts"])
        csv_line["% of alerts compared to the baseline"] =  len(data["alerts"])/len(baseline_data[key]["alerts"])
        csv_line["Packets cloned to NIDS"] = data["packets_redirected"]
        csv_line["% of packets cloned to NIDS compared to the baseline"] =  \
                 data["packets_redirected"]/baseline_packets_redirected[folder_name_elements[3]][key]
        csv_data.append(csv_line)

keys = csv_data[0].keys()
output_file = output_folder + "/final_evaluation.csv"
with open(output_file, 'w') as file:
    w = csv.DictWriter(file, keys)
    w.writeheader()

    for line in csv_data:
        w.writerow(line)

### Importing data mining and plotting libraries

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


parameters_evaluation_df = pd.read_csv(output_file, sep=",")
parameters_evaluation_df

,PCAP,Topology,Offloading algorithm,Available space (%),Alerts,% of alerts compared to the baseline,Packets cloned to NIDS,% of packets cloned to NIDS compared to the baseline
0,Tuesday,linear,bestfit,100,85925,1.023648,4137215,0.358140
1,Thursday,linear,bestfit,100,58441,0.786819,2815171,0.301991
2,Wednesday,linear,bestfit,100,194904,0.469543,9886410,0.716984
3,Monday,linear,bestfit,100,60190,0.913312,2892670,0.247026
4,Friday,linear,bestfit,100,64694,0.957437,5439055,0.544021
5,Tuesday,linear,simple,100,97017,1.155790,2177894,0.188530
6,Thursday,linear,simple,100,52757,0.710293,720208,0.077259
7,Wednesday,linear,simple,100,174994,0.421578,5500008,0.398873
8,Monday,linear,simple,100,59524,0.903206,828228,0.070728
9,Friday,linear,simple,100,60451,0.894643,2131607,0.213206
